In [1]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from NesrHydrusAnalyst import *

# Comparing files
- /Nesr/1-Original_Grid.csv
- /Nesr/2-Original_PrettyGrid.csv
- /Nesr/3-Atm_A_level.csv
- /Nesr/4-MassBalance.csv
- /Nesr/5-Simulation_info.csv
- /Nesr/6-BoundaryData.csv
- /Nesr/7-MaterialsData.csv

In [2]:
sources = [
    f'../Datasets/H3D2_SandDitch0014{x}' for x in ('', 'a', 'b', 'c', 'd', 'e')
]

## Working with `2-Original_PrettyGrid.csv`

In [3]:
file_name = '2-Original_PrettyGrid.csv'
orig_df = pd.read_csv(os.path.join(sources[0], 'Nesr', file_name))
comp_df = pd.read_csv(os.path.join(sources[1], 'Nesr', file_name))
orig_df.head()

,n,x,y,z,Time,H,Th,V1,V2,V3
0,1,7.5,30.0,19.25,0,-2104.020,0.150000,-2.814030e-08,0.000001,-0.000010
1,1,7.5,30.0,19.25,5,-2090.340,0.150278,-1.781770e-08,0.000001,-0.000010
2,1,7.5,30.0,19.25,15,-2064.730,0.150805,1.920960e-09,0.000002,-0.000011
3,1,7.5,30.0,19.25,30,-2028.670,0.151563,3.029510e-08,0.000002,-0.000011
4,1,7.5,30.0,19.25,60,-428.565,0.239586,-2.794120e-04,-0.000006,-0.000831


In [4]:
matching=(orig_df==comp_df).mean()*100
matching[matching<100]

H     10.081603
Th    15.309587
V1     5.813513
V2     5.802684
V3     5.887767
dtype: float64

In [5]:
orig_df = pd.read_csv(os.path.join(sources[0], 'Nesr', file_name))
results = {}
for other in range(1, len(sources)):
    source = sources[other]
    comp_df = pd.read_csv(os.path.join(source, 'Nesr', file_name))
    matching = (orig_df == comp_df).mean() * 100
    results[source.split("/")[2][-3:]] = matching

results = pd.DataFrame(results)
results[results < 100].dropna()

,14a,14b,14c,14d,14e
H,10.081603,6.276830,9.125575,6.773408,6.442356
Th,15.309587,7.231311,10.414201,7.974630,7.518274
V1,5.813513,5.316162,5.347101,5.313068,5.313842
V2,5.802684,5.326217,5.336273,5.323123,5.306880
V3,5.887767,5.269753,5.337046,5.280582,5.272073


## Working with `3-Atm_A_level.csv`

In [6]:
file_name = '3-Atm_A_level.csv'
orig_df = pd.read_csv(os.path.join(sources[0], 'Nesr', file_name))
comp_df = pd.read_csv(os.path.join(sources[1], 'Nesr', file_name))
orig_df.head()

,Time,CumQAP,CumQRP,CumQA,CumQR,CumQ3,hAtm,hRoot,hKode3,A-level,...,rRoot,hCritA,rt,ht,rt.1,ht.1,rt.2,ht.2,rt.3,ht.3
0,60.0,0.0,0.0,0.0,0.0,-4050.0,-1.8,0.0,8.0,1,...,0.000278,10000,-1,0,0,0,0,0,0,0
1,2880.0,0.0,0.0,0.0,0.0,-4050.0,-39.8,0.0,-39.5,2,...,0.000278,10000,0,0,0,0,0,0,0,0


In [7]:
matching=(orig_df==comp_df).mean()*100

In [8]:
matching[matching<100]

hAtm      0.0
hKode3    0.0
dtype: float64

In [9]:
orig_df = pd.read_csv(os.path.join(sources[0], 'Nesr', file_name))
results = {}
for other in range(1,len(sources)):
    source = sources[other]
    comp_df = pd.read_csv(os.path.join(source, 'Nesr', file_name))
    matching=(orig_df==comp_df).mean()*100
    results[source.split("/")[2][-3:]]=matching

results = pd.DataFrame(results)
results[results<100].dropna()

,14a,14b,14c,14d,14e
hAtm,0.0,0.0,0.0,0.0,0.0
hKode3,0.0,0.0,0.0,0.0,0.0


In [10]:
falsy = orig_df != comp_df
orig_f, comp_f = (orig_df[falsy].dropna(axis=1, how='all'),
                  comp_df[falsy].dropna(axis=1, how='all'))
display(orig_f, comp_f )

,hAtm,hKode3
0,-1.8,8.0
1,-39.8,-39.5


,hAtm,hKode3
0,-8.6,-6.4
1,-42.9,-42.5


In [11]:
def convert_df_to_series(df, sep='_'):
    # df= orig_df
    # sep='_'
    ser={}
    for col in list(df):
        for row in df.index:
            nam = f'{col}{sep}{row}'
            val = df.loc[row][col]
            ser[nam]=val
    return pd.Series(ser)

In [12]:
# Test
convert_df_to_series(orig_f)    

hAtm_0      -1.8
hAtm_1     -39.8
hKode3_0     8.0
hKode3_1   -39.5
dtype: float64

In [13]:
def display_difference_values_0(the_file_name):
    orig_df = pd.read_csv(os.path.join(sources[0], 'Nesr', the_file_name))
    results = {}
    for other in range(1, len(sources)):
        source = sources[other]
        comp_df = pd.read_csv(os.path.join(source, 'Nesr', the_file_name))
        falsy = orig_df != comp_df
        orig_f = convert_df_to_series(orig_df[falsy].dropna(axis=1, how='all'))
        comp_f = convert_df_to_series(comp_df[falsy].dropna(axis=1, how='all'))
        if '14_' in results:
            #display(results['14_'].index , orig_f.index)
            if list(results['14_'].index) != list(orig_f.index):
                print('There is an error here')
        else:
            results['14_'] = orig_f

        results[source.split("/")[2][-3:]] = comp_f

    results = pd.DataFrame(results).dropna()
    return results

In [14]:
display_difference_values_0(file_name)

,14_,14a,14b,14c,14d,14e
hAtm_0,-1.8,0.6,-6.4,-7.1,-8.3,-8.6
hAtm_1,-39.8,-39.5,-43.1,-41.8,-43.7,-42.9
hKode3_0,8.0,9.5,-4.3,-4.6,-6.3,-6.4
hKode3_1,-39.5,-39.3,-42.7,-41.5,-43.2,-42.5


## Working with `4-MassBalance.csv`

In [15]:
file_name = '4-MassBalance.csv'

In [16]:
orig_df = pd.read_csv(os.path.join(sources[0], 'Nesr', file_name))
comp_df = pd.read_csv(os.path.join(sources[1], 'Nesr', file_name))
orig_df.head()

,Time,Volume,VolumeW,InFlow,hMean,WatBalT,WatBalR
0,0.0,37804.0,5670.0,0.000,-1419.200,NaN,NaN
1,5.0,37804.0,6008.2,67.482,-465.180,0.76695,0.087
2,15.0,37804.0,6683.1,67.487,-380.310,0.72489,0.043
3,30.0,37804.0,7695.6,67.500,-264.770,0.74317,0.028
4,60.0,37804.0,9720.5,67.487,-93.056,0.65117,0.014


In [17]:
matching=(orig_df==comp_df).mean()*100

In [18]:
matching[matching<100]

VolumeW    7.692308
InFlow     7.692308
hMean      0.000000
WatBalT    0.000000
WatBalR    0.000000
dtype: float64

In [19]:
orig_df = pd.read_csv(os.path.join(sources[0], 'Nesr', file_name))
results = {}
for other in range(1,len(sources)):
    source = sources[other]
    comp_df = pd.read_csv(os.path.join(source, 'Nesr', file_name))
    matching=(orig_df==comp_df).mean()*100
    results[source.split("/")[2][-3:]]=matching

results = pd.DataFrame(results)
results[results<100].dropna()

,14a,14b,14c,14d,14e
VolumeW,7.692308,0.000000,0.000000,0.000000,0.000000
InFlow,7.692308,7.692308,7.692308,15.384615,15.384615
hMean,0.000000,0.000000,0.000000,0.000000,0.000000
WatBalT,0.000000,0.000000,0.000000,0.000000,0.000000
WatBalR,0.000000,0.000000,7.692308,0.000000,0.000000


In [20]:
display_difference_values_0(file_name)

,14_,14a,14b,14c,14d,14e
VolumeW_1,6008.200000,6007.600000,5761.100000,5761.300000,5761.700000,5761.300000
VolumeW_2,6683.100000,6682.500000,6435.700000,6436.200000,6436.500000,6436.200000
VolumeW_3,7695.600000,7695.000000,7447.500000,7448.500000,7448.800000,7448.700000
VolumeW_4,9720.500000,9675.200000,9195.100000,9427.300000,9230.600000,9363.300000
VolumeW_5,9708.400000,9620.100000,9056.500000,9279.000000,9019.700000,9145.500000
VolumeW_6,9708.200000,9619.900000,9055.600000,9277.000000,9016.800000,9141.000000
VolumeW_7,9708.200000,9619.800000,9055.300000,9276.400000,9016.000000,9139.700000
VolumeW_8,9708.200000,9619.700000,9054.200000,9274.800000,9013.500000,9136.500000
VolumeW_9,9708.000000,9619.600000,9054.600000,9273.800000,9011.800000,9134.400000
VolumeW_10,9707.600000,9619.300000,9055.200000,9272.100000,9011.700000,9131.100000


## Working with `5-Simulation_info.csv`

In [21]:
file_name = '5-Simulation_info.csv'

In [22]:
orig_df = pd.read_csv(os.path.join(
    sources[0], 'Nesr', file_name)).rename(columns={
        'Unnamed: 0': "Feature",
        'Sand Ditch simulation': "Value"
    }).set_index('Feature')
comp_df = pd.read_csv(os.path.join(
    sources[1], 'Nesr', file_name)).rename(columns={
        'Unnamed: 0': "Feature",
        'Sand Ditch simulation': "Value"
    }).set_index('Feature')
orig_df.Value.rename_axis(None).head()

SimulTime_s    51.03
L_Unit          1.00
T_Unit          1.00
Category        3.00
MaxIt          10.00
Name: Value, dtype: float64

In [23]:
matching = (orig_df == comp_df).mean() * 100
matching[matching<100]

Value    68.503937
dtype: float64

In [24]:
orig_df = pd.read_csv(os.path.join(
    sources[0], 'Nesr', file_name)).rename(columns={
        'Unnamed: 0': "Feature",
        'Sand Ditch simulation': "Value"
    }).set_index('Feature')
results = {}
for other in range(1, len(sources)):
    source = sources[other]
    comp_df = pd.read_csv(os.path.join(
        source, 'Nesr', file_name)).rename(columns={
            'Unnamed: 0': "Feature",
            'Sand Ditch simulation': "Value"
        }).set_index('Feature')
    matching = (orig_df == comp_df).mean() * 100
    results[source.split("/")[2][-3:]] = matching

results = pd.DataFrame(results)
results[results < 100].dropna()

,14a,14b,14c,14d,14e
Value,68.503937,67.322835,67.716535,67.716535,67.716535


In [25]:
display_difference_values_0(file_name)

,14_,14a,14b,14c,14d,14e
Sand Ditch simulation_0,51.030,41.760,62.850,45.150,41.970,47.080
Sand Ditch simulation_44,0.045,0.010,0.010,0.010,0.010,0.010
Sand Ditch simulation_45,0.430,0.300,0.300,0.530,0.530,0.680
Sand Ditch simulation_46,0.145,0.151,0.151,0.151,0.151,0.151
Sand Ditch simulation_47,2.680,2.750,2.750,2.750,2.750,2.750
...,...,...,...,...,...,...
Sand Ditch simulation_196,-25.838,-25.520,-29.146,-27.845,-29.728,-28.990
Sand Ditch simulation_202,0.045,0.010,0.010,0.010,0.010,0.010
Sand Ditch simulation_211,0.430,0.300,0.300,0.530,0.530,0.680
Sand Ditch simulation_220,0.145,0.151,0.151,0.151,0.151,0.151


## Working with `6-BoundaryData.csv`

In [26]:
file_name = '6-BoundaryData.csv'

In [27]:
orig_df = pd.read_csv(os.path.join(sources[0], 'Nesr', file_name))
comp_df = pd.read_csv(os.path.join(sources[1], 'Nesr', file_name))
display(orig_df.head(), comp_df.head())

,Time_T,CumQAP_L3,CumQRP_L3,CumQA_L3,CumQR_L3,CumQ3_L3,CumQ1_L3,CumQS_L3,CumQ5_L3,CumQ6_L3,...,vKode7_L3pT,vKode8_L3pT,vKode9_L3pT,RunOff_LpT,Evapor_LpT,Infiltr_LpT,SnowLayer_L,dt,Iter,ItCum
0,0.0010,0.0,0.0,0.0,0.0,-0.0675,0.0,0.0,0.0,0.000188,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001000,2,2
1,0.0020,0.0,0.0,0.0,0.0,-0.1350,0.0,0.0,0.0,0.000371,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001000,2,4
2,0.0033,0.0,0.0,0.0,0.0,-0.2230,0.0,0.0,0.0,0.000601,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001300,2,6
3,0.0050,0.0,0.0,0.0,0.0,-0.3370,0.0,0.0,0.0,0.000889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001690,3,9
4,0.0072,0.0,0.0,0.0,0.0,-0.4850,0.0,0.0,0.0,0.001250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002197,3,12


,Time_T,CumQAP_L3,CumQRP_L3,CumQA_L3,CumQR_L3,CumQ3_L3,CumQ1_L3,CumQS_L3,CumQ5_L3,CumQ6_L3,...,vKode7_L3pT,vKode8_L3pT,vKode9_L3pT,RunOff_LpT,Evapor_LpT,Infiltr_LpT,SnowLayer_L,dt,Iter,ItCum
0,0.0010,0.0,0.0,0.0,0.0,-0.0675,0.0,0.0,0.0,0.00178,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00100,5,5
1,0.0020,0.0,0.0,0.0,0.0,-0.1350,0.0,0.0,0.0,0.00327,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00100,4,9
2,0.0030,0.0,0.0,0.0,0.0,-0.2030,0.0,0.0,0.0,0.00458,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00100,3,12
3,0.0043,0.0,0.0,0.0,0.0,-0.2900,0.0,0.0,0.0,0.00610,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00130,3,15
4,0.0060,0.0,0.0,0.0,0.0,-0.4040,0.0,0.0,0.0,0.00787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00169,3,18


In [28]:
list(orig_df)== list(comp_df)

True

In [29]:
list(orig_df.index)== list(comp_df.index)

False

In [30]:
[(a, b) for a,b in zip(orig_df.index,comp_df.index) if a!=b]

[]

In [31]:
orig_df.shape, comp_df.shape

((402, 45), (385, 45))

In [32]:
print('14_', orig_df.shape)
for other in range(1,len(sources)):
    source = sources[other]
    comp_df = pd.read_csv(os.path.join(source, 'Nesr', file_name))
    print(f'{source.split("/")[2][-3:]}', comp_df.shape)

14_ (402, 45)
14a (385, 45)
14b (384, 45)
14c (389, 45)
14d (360, 45)
14e (365, 45)


In [33]:
# matching=(orig_df==comp_df).mean()*100

In [34]:
# matching[matching<100]

In [35]:
# orig_df = pd.read_csv(os.path.join(sources[0], 'Nesr', file_name))
# results = {}
# for other in range(1,len(sources)):
#     source = sources[other]
#     comp_df = pd.read_csv(os.path.join(source, 'Nesr', file_name))
#     matching=(orig_df==comp_df).mean()*100
#     results[source.split("/")[2][-3:]]=matching

# results = pd.DataFrame(results)
# results[results<100].dropna()

## Working with `7-MaterialsData.csv`

In [36]:
file_name = '7-MaterialsData.csv'

In [37]:
orig_df = pd.read_csv(os.path.join(sources[0], 'Nesr', file_name))
comp_df = pd.read_csv(os.path.join(sources[1], 'Nesr', file_name))
orig_df.head()

,Material,theta,h,log_h,C,K,log_K,S
0,1,0.43,-0.0100,-2.000,0.0,0.495,-0.3054,1.0
1,1,0.43,-0.0115,-1.939,0.0,0.495,-0.3054,1.0
2,1,0.43,-0.0132,-1.879,0.0,0.495,-0.3054,1.0
3,1,0.43,-0.0152,-1.818,0.0,0.495,-0.3054,1.0
4,1,0.43,-0.0175,-1.758,0.0,0.495,-0.3054,1.0


In [38]:
matching=(orig_df==comp_df).mean()*100

In [39]:
matching[matching<100]

theta    88.914027
C        92.986425
K        92.986425
log_K    92.986425
S        94.570136
dtype: float64

In [40]:
orig_df = pd.read_csv(os.path.join(sources[0], 'Nesr', file_name))
results = {}
for other in range(1,len(sources)):
    source = sources[other]
    comp_df = pd.read_csv(os.path.join(source, 'Nesr', file_name))
    matching=(orig_df==comp_df).mean()*100
    results[source.split("/")[2][-3:]]=matching

results = pd.DataFrame(results)
results[results<100].dropna()

,14a,14b,14c,14d,14e
theta,88.914027,88.914027,88.914027,88.914027,88.914027
C,92.986425,92.986425,92.986425,92.986425,92.986425
K,92.986425,88.914027,88.914027,88.914027,88.914027
log_K,92.986425,88.914027,88.914027,88.914027,88.914027
S,94.570136,94.570136,94.570136,94.570136,94.570136


In [41]:
display_difference_values_0(file_name)

,14_,14a,14b,14c,14d,14e
theta_198,0.4300,0.3000,0.3000,0.5300,0.5300,0.6800
theta_199,0.4300,0.3000,0.3000,0.5300,0.5300,0.6800
theta_200,0.4300,0.3000,0.3000,0.5300,0.5300,0.6800
theta_201,0.4300,0.3000,0.3000,0.5300,0.5300,0.6800
theta_202,0.4300,0.3000,0.3000,0.5300,0.5300,0.6800
...,...,...,...,...,...,...
S_278,0.0003,0.0002,0.0002,0.0002,0.0002,0.0002
S_279,0.0002,0.0001,0.0001,0.0001,0.0001,0.0001
S_280,0.0002,0.0001,0.0001,0.0001,0.0001,0.0001
S_284,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000


# Generalized functions

In [42]:
sources = [
    f'../Datasets/H3D2_SandDitch0014{x}' for x in ('', 'a', 'b', 'c', 'd', 'e')
]
file_names = ('2-Original_PrettyGrid.csv', '3-Atm_A_level.csv',
              '4-MassBalance.csv', '5-Simulation_info.csv',
              '6-BoundaryData.csv', '7-MaterialsData.csv')

In [46]:
def get_similarity_percent(file_name,
                           original,
                           others,
                           caption_length=3,
                           normal_csv=True):
    '''
    getting a dataframe (table) of the percent which some files in `others` 
    folder list differ from the `original` folder. the comparisons occur only
    on the specified `file_name`. The returned table's header contains part
    of the others files, which is limited to the last `caption_length` letters
    (Default=3 letters), for example, if the folder name is Hydrus_sand_exp15e
    then the caption of this will be 15e if the caption_length=3
    if normal_csv=False, then the file need special read function
    '''
    try:
        if normal_csv:
            orig_df = pd.read_csv(os.path.join(original, 'Nesr', file_name))
        else:
            orig_df = pd.read_csv(os.path.join(
                sources[0], 'Nesr',
                file_name)).rename(columns={
                    'Unnamed: 0': "Feature",
                    'Sand Ditch simulation': "Value"
                }).set_index('Feature')
        results = {}
        for source in others:
            if normal_csv:
                comp_df = pd.read_csv(os.path.join(source, 'Nesr', file_name))
            else:
                comp_df = pd.read_csv(os.path.join(
                    source, 'Nesr',
                    file_name)).rename(columns={
                        'Unnamed: 0': "Feature",
                        'Sand Ditch simulation': "Value"
                    }).set_index('Feature')

            matching = (orig_df == comp_df).mean() * 100
            results[source.split("/")[2][-caption_length:]] = matching
        results = pd.DataFrame(results)
        return results[results < 100].dropna()

    except:
        print(
                f'The difference percent table cannot be calculated.')

            


# Test
get_similarity_percent(file_names[0], sources[0], sources[1:])

,14a,14b,14c,14d,14e
H,10.081603,6.276830,9.125575,6.773408,6.442356
Th,15.309587,7.231311,10.414201,7.974630,7.518274
V1,5.813513,5.316162,5.347101,5.313068,5.313842
V2,5.802684,5.326217,5.336273,5.323123,5.306880
V3,5.887767,5.269753,5.337046,5.280582,5.272073


In [48]:
# Test
for f in file_names:
    print (f"Similarity percent of {f}")
    display(get_similarity_percent(f, sources[0], sources[1:]))
    print ("====================================")

Similarity percent of 2-Original_PrettyGrid.csv


,14a,14b,14c,14d,14e
H,10.081603,6.276830,9.125575,6.773408,6.442356
Th,15.309587,7.231311,10.414201,7.974630,7.518274
V1,5.813513,5.316162,5.347101,5.313068,5.313842
V2,5.802684,5.326217,5.336273,5.323123,5.306880
V3,5.887767,5.269753,5.337046,5.280582,5.272073


Similarity percent of 3-Atm_A_level.csv


,14a,14b,14c,14d,14e
hAtm,0.0,0.0,0.0,0.0,0.0
hKode3,0.0,0.0,0.0,0.0,0.0


Similarity percent of 4-MassBalance.csv


...

,14a,14b,14c,14d,14e
Sand Ditch simulation,68.503937,67.322835,67.716535,67.716535,67.716535


Similarity percent of 6-BoundaryData.csv
The difference percent table cannot be calculated.


None

Similarity percent of 7-MaterialsData.csv


,14a,14b,14c,14d,14e
theta,88.914027,88.914027,88.914027,88.914027,88.914027
C,92.986425,92.986425,92.986425,92.986425,92.986425
K,92.986425,88.914027,88.914027,88.914027,88.914027
log_K,92.986425,88.914027,88.914027,88.914027,88.914027
S,94.570136,94.570136,94.570136,94.570136,94.570136


In [53]:
def display_difference_values(the_file_name,
                              original,
                              others,
                              caption_length=3):
    try:
        orig_df = pd.read_csv(os.path.join(original, 'Nesr', the_file_name))
        results = {}
        for source in others:
            comp_df = pd.read_csv(os.path.join(source, 'Nesr', the_file_name))
            falsy = orig_df != comp_df
            orig_f = convert_df_to_series(orig_df[falsy].dropna(axis=1, how='all'))
            comp_f = convert_df_to_series(comp_df[falsy].dropna(axis=1, how='all'))
            if '14_' in results:
                if list(results['14_'].index) != list(orig_f.index):
                    print('There is an error here')
            else:
                results['14_'] = orig_f

            results[source.split("/")[2][-caption_length:]] = comp_f

        results = pd.DataFrame(results).dropna()
        return results
    except:
        print(
                f'The difference values table cannot be calculated.')

In [54]:
# Test
for f in file_names[1:]:
    print (f"\nDifference values of {f}")
    display(display_difference_values(f, sources[0], sources[1:]))
    print ("====================================")


Difference values of 3-Atm_A_level.csv


,14_,14a,14b,14c,14d,14e
hAtm_0,-1.8,0.6,-6.4,-7.1,-8.3,-8.6
hAtm_1,-39.8,-39.5,-43.1,-41.8,-43.7,-42.9
hKode3_0,8.0,9.5,-4.3,-4.6,-6.3,-6.4
hKode3_1,-39.5,-39.3,-42.7,-41.5,-43.2,-42.5



Difference values of 4-MassBalance.csv


,14_,14a,14b,14c,14d,14e
VolumeW_1,6008.200000,6007.600000,5761.100000,5761.300000,5761.700000,5761.300000
VolumeW_2,6683.100000,6682.500000,6435.700000,6436.200000,6436.500000,6436.200000
VolumeW_3,7695.600000,7695.000000,7447.500000,7448.500000,7448.800000,7448.700000
VolumeW_4,9720.500000,9675.200000,9195.100000,9427.300000,9230.600000,9363.300000
VolumeW_5,9708.400000,9620.100000,9056.500000,9279.000000,9019.700000,9145.500000
VolumeW_6,9708.200000,9619.900000,9055.600000,9277.000000,9016.800000,9141.000000
VolumeW_7,9708.200000,9619.800000,9055.300000,9276.400000,9016.000000,9139.700000
VolumeW_8,9708.200000,9619.700000,9054.200000,9274.800000,9013.500000,9136.500000
VolumeW_9,9708.000000,9619.600000,9054.600000,9273.800000,9011.800000,9134.400000
VolumeW_10,9707.600000,9619.300000,9055.200000,9272.100000,9011.700000,9131.100000



Difference values of 5-Simulation_info.csv


,14_,14a,14b,14c,14d,14e
Sand Ditch simulation_0,51.030,41.760,62.850,45.150,41.970,47.080
Sand Ditch simulation_44,0.045,0.010,0.010,0.010,0.010,0.010
Sand Ditch simulation_45,0.430,0.300,0.300,0.530,0.530,0.680
Sand Ditch simulation_46,0.145,0.151,0.151,0.151,0.151,0.151
Sand Ditch simulation_47,2.680,2.750,2.750,2.750,2.750,2.750
...,...,...,...,...,...,...
Sand Ditch simulation_196,-25.838,-25.520,-29.146,-27.845,-29.728,-28.990
Sand Ditch simulation_202,0.045,0.010,0.010,0.010,0.010,0.010
Sand Ditch simulation_211,0.430,0.300,0.300,0.530,0.530,0.680
Sand Ditch simulation_220,0.145,0.151,0.151,0.151,0.151,0.151



Difference values of 6-BoundaryData.csv
The difference values table cannot be calculated.


None


Difference values of 7-MaterialsData.csv


,14_,14a,14b,14c,14d,14e
theta_198,0.4300,0.3000,0.3000,0.5300,0.5300,0.6800
theta_199,0.4300,0.3000,0.3000,0.5300,0.5300,0.6800
theta_200,0.4300,0.3000,0.3000,0.5300,0.5300,0.6800
theta_201,0.4300,0.3000,0.3000,0.5300,0.5300,0.6800
theta_202,0.4300,0.3000,0.3000,0.5300,0.5300,0.6800
...,...,...,...,...,...,...
S_278,0.0003,0.0002,0.0002,0.0002,0.0002,0.0002
S_279,0.0002,0.0001,0.0001,0.0001,0.0001,0.0001
S_280,0.0002,0.0001,0.0001,0.0001,0.0001,0.0001
S_284,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000
